In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord, GeocentricMeanEcliptic
import astropy.units as u
from io import StringIO
import sys
import re
from astropy.time import Time
import datetime
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, GCRS, get_body
import math

In [2]:
def get_az(row):
    ra = row["ra"]
    dec = row["dec"]
    ra_formatted = ra.split()[0] + "h" + ra.split()[1] + "m" + ra.split()[2] + "s"
    dec_formatted = dec.split()[0] + "d" + dec.split()[1] + "m" + dec.split()[2] + "s"
    c = SkyCoord(ra_formatted, dec_formatted, frame='icrs')

    lon = row["CenterLon"]
    lat = row["CenterLat"]
    earth_loc = EarthLocation(lat=lat, lon=lon, height=400*u.m)
    time = row["time"]
    datetime_obj = datetime.datetime.strptime(time.strip(), '%Y-%b-%d %H:%M')
    time = Time(str(datetime_obj))
    c_horiz = c.transform_to(AltAz(obstime=time,location=earth_loc))
    return c_horiz.az *u.deg

def get_alt(row):
    ra = row["ra"]
    dec = row["dec"]
    ra_formatted = ra.split()[0] + "h" + ra.split()[1] + "m" + ra.split()[2] + "s"
    dec_formatted = dec.split()[0] + "d" + dec.split()[1] + "m" + dec.split()[2] + "s"
    c = SkyCoord(ra_formatted, dec_formatted, frame='icrs')

    lon = row["CenterLon"]
    lat = row["CenterLat"]
    earth_loc = EarthLocation(lat=lat, lon=lon, height=400*u.m)
    time = row["time"]
    datetime_obj = datetime.datetime.strptime(time.strip(), '%Y-%b-%d %H:%M')
    time = Time(str(datetime_obj))
    c_horiz = c.transform_to(AltAz(obstime=time,location=earth_loc))
    return c_horiz.alt *u.deg

def get_dist(row):
    ra = row["ra"]
    dec = row["dec"]
    ra_formatted = ra.split()[0] + "h" + ra.split()[1] + "m" + ra.split()[2] + "s"
    dec_formatted = dec.split()[0] + "d" + dec.split()[1] + "m" + dec.split()[2] + "s"
    c = SkyCoord(ra_formatted, dec_formatted, frame='gcrs')
    
    lon = row["CenterLon"]
    lat = row["CenterLat"]
    earth_loc = EarthLocation(lat=lat, lon=lon, height=400*u.m)
    time = row["time"]
    datetime_obj = datetime.datetime.strptime(time.strip(), '%Y-%b-%d %H:%M')
    time = Time(str(datetime_obj))
    planet = get_body(row["Planet"].lower(), time)
    earth = get_body('earth', time, earth_loc)
    return earth.separation_3d(planet)

def get_earth_loc(row):
    lon = row["CenterLon"]
    lat = row["CenterLat"]
    earth_loc = EarthLocation(lat=lat, lon=lon, height=400*u.m)
    time = row["time"]
    datetime_obj = datetime.datetime.strptime(time.strip(), '%Y-%b-%d %H:%M')
    time = Time(str(datetime_obj))
    earth = get_body('earth', time, earth_loc)
    return (earth.cartesian.x, earth.cartesian.y, earth.cartesian.z)


def get_planet_loc(row):
    lon = row["CenterLon"]
    lat = row["CenterLat"]
    earth_loc = EarthLocation(lat=lat, lon=lon, height=400*u.m)
    time = row["time"]
    datetime_obj = datetime.datetime.strptime(time.strip(), '%Y-%b-%d %H:%M')
    time = Time(str(datetime_obj))
    planet = get_body(row["Planet"].lower(), time)
    return (planet.cartesian.x, planet.cartesian.y, planet.cartesian.z)

In [4]:
##construct name dictionaries
invalid_loc = set()
num_to_planet = {}
for i in range(1, 9):
    output_filename = f"output{i}_{1}.txt"
    f = open(output_filename, "r")
    string = f.read()
    string = string.replace("\\n", "").replace("\\r", "\n")
    string = string.split("\n")
    if i != 9:
        num_to_planet[i] = string[1].split()[4]
    else:
        num_to_planet[i] = string[1].split()[5]
    f.close()
num_to_location = {}
for i in range(1, 999):
    output_filename = f"output{1}_{i}.txt"
    f = open(output_filename, "r")
    string = f.read()
    string = string.replace("\\n", "").replace("\\r", "\n")
    string = string.split("\n");
    
    r = re.compile("^Center-site name")
    newlist = list(filter(r.match, string))
    if newlist:
        placeline = newlist[0].split()
        name = ""
        for j in range(2, len(placeline)):
            name = name + placeline[j]
        name = name.replace(",", " ")
        num_to_location[i] = name
    else:
        invalid_loc.add(i)
    f.close()
num_to_position = {}
for i in range(1, 999):
    output_filename = f"output{1}_{i}.txt"
    f = open(output_filename, "r")
    string = f.read()
    string = string.replace("\\n", "").replace("\\r", "\n")
    string = string.split("\n");
    r = re.compile("^Center geodetic")
    newlist = list(filter(r.match, string))
    if newlist:
        lon = newlist[0].split()[3].split(",")[0]
        lat = newlist[0].split()[3].split(",")[1]
        num_to_position[i] = (lon, lat)
    else:
        invalid_loc.add(i)


In [5]:
def generate_df(num_locations, num_days=5, time_step=1, planet=[], location=[]):
    final_string = []
    for i in range(1, 9):
        if planet:
            if i not in planet:
                continue
        for j in range(1, num_locations):
            if location:
                if j not in location:
                    continue
            if j in invalid_loc:
                continue
            output_filename = f"output{i}_{j}.txt"
            f = open(output_filename, "r")
            string = f.read()
            string = string.replace("\\n", "").replace("\\r", "\n")
            string = string.split("\n")
            if (final_string == []):
                categories_ind = -1
                for index, elem in enumerate(string):
                    if elem.find("Date__(UT)__HR:MN") != -1:
                        categories_ind = index
                        break
                if categories_ind != -1:
                    final_string = [string[categories_ind] + ",Planet, CenterName, CenterLon, CenterLat"]
            data_ind = -1
            for index, elem in enumerate(string):
                if elem.find("Date__(UT)__HR:MN") != -1:
                    data_ind = index + 1
                    break
            if data_ind != -1:
                string = string[data_ind:data_ind + num_days:time_step]
                string = [x + "," + num_to_planet[i] + "," + num_to_location[j] + "," + num_to_position[j][0] + "," + num_to_position[j][1] for x in string if len(x.split(",")) == 13]
                final_string += string
    df = pd.DataFrame([x.split(',') for x in final_string[1:]], columns=[x for x in final_string[0].split(',')])
    df.drop([df.columns[1], df.columns[2], df.columns[5], df.columns[6], df.columns[7], df.columns[8], df.columns[9], df.columns[10], df.columns[11], df.columns[12]], axis=1, inplace=True)
    df.columns = [x.strip() for x in df.columns]
    df = df.rename(columns={"Date__(UT)__HR:MN" : "time","R.A._(ICRF)": "ra", "DEC__(ICRF)": "dec"})
    df['az'] = df.apply(lambda row: get_az(row), axis = 1)
    df['alt'] = df.apply(lambda row: get_alt(row), axis = 1)
    df['dist (center to planet AU)'] = df.apply(lambda row: get_dist(row), axis = 1)
    df['Center_XYZ (Geocentric AU)'] = df.apply(lambda row: get_earth_loc(row), axis = 1)
    df['Planet_XYZ (Geocentric AU)'] = df.apply(lambda row: get_planet_loc(row), axis = 1)
    return df

### Run cell below to generate pandas dataframe with desired values
generate_df can be called with a given list of locations, and a given list of planets, which are defined by indices from 1-1000 and 1-9 respectively, mappings for which are defined by the num_to_location and num_to_planet dictionaries above, which map from indices to location/planet names. generate_df also takes in a number of days and a time step for positional data to be returned, starting from Jan 01 2000, and going until Dec 25 2020 (max number of days is ~7000 and min time step is 1 day, with time step being restricted to integer values only)

In [14]:
##EXAMPLE
##generate_df(num_locations=10, num_days=5, time_step=1, planet=[1, 8], location=[1])
##The above line of code will call generate_df and return a dataframe with positional data for mercury and neptune
##centered at the location with index 1 in num_to_location (which is Crowborough)
##If planet and location are left as empty lists, the function will default to returning data for every planet and every location
df = generate_df(10, num_days=5, time_step=1, planet=[1, 8], location=[1])
df

,time,ra,dec,Planet,CenterName,CenterLon,CenterLat,az,alt,dist (center to planet AU),Center_XYZ (Geocentric AU),Planet_XYZ (Geocentric AU)
0,2000-Jan-01 00:00,18 04 55.50,-24 22 51.5,Mercury,Crowborough,0.15420000,51.0518512,17.74343784755384 deg2,-62.460788027094935 deg2,1.4130914988961565 AU,"(4.721613862569572e-06 AU, -2.643813340466896e...","(0.027512728560183226 AU, -1.2867867380646507 ..."
1,2000-Jan-02 00:00,18 11 45.95,-24 27 21.3,Mercury,Crowborough,0.15420000,51.0518512,16.374140650186238 deg2,-62.66790959130999 deg2,1.4177089536661651 AU,"(5.175688925124622e-06 AU, -2.6353003563308614...","(0.06607983243067013 AU, -1.2888263257485577 A..."
2,2000-Jan-03 00:00,18 18 38.19,-24 30 32.3,Mercury,Crowborough,0.15420000,51.0518512,14.965991636726711 deg2,-62.84443967309148 deg2,1.421780515121529 AU,"(5.628230511401381e-06 AU, -2.6260075568148384...","(0.10493844330204292 AU, -1.2894158685079364 A..."
3,2000-Jan-01 00:00,20 21 41.83,-19 12 59.3,Neptune,Crowborough,0.15420000,51.0518512,318.56401634757674 deg2,-52.43516868446793 deg2,31.021028239772164 AU,"(4.721613862569572e-06 AU, -2.643813340466896e...","(16.976404171181013 AU, -23.871433537342604 AU..."
4,2000-Jan-02 00:00,20 21 50.64,-19 12 30.5,Neptune,Crowborough,0.15420000,51.0518512,319.8992216569468 deg2,-52.81749185318781 deg2,31.027733371983413 AU,"(5.175688925124622e-06 AU, -2.6353003563308614...","(16.99617452573775 AU, -23.866878985531233 AU,..."
5,2000-Jan-03 00:00,20 21 59.50,-19 12 01.6,Neptune,Crowborough,0.15420000,51.0518512,321.25414539949503 deg2,-53.18894639299184 deg2,31.034166715540692 AU,"(5.628230511401381e-06 AU, -2.6260075568148384...","(17.015888621116655 AU, -23.862046956092218 AU..."


In [9]:
### START HERE
planet_events = pd.read_csv('planets.csv', skiprows=1)
planet_events.columns = ['Year', 'Month', 'Date', 'Day', 'Time', 'Event', 'NaNdos']
planet_events['Year'] = planet_events['Year'].fillna(method='ffill').astype(int).astype(str)
planet_events['Month'] = planet_events['Month'].fillna(method='ffill')
planet_events['Date'] = planet_events['Date'].astype(str)
planet_events['Datetime'] = planet_events['Month'] + ' ' + planet_events['Date'] + ' ' + planet_events['Year']
planet_events['Datetime'] = pd.to_datetime(planet_events['Datetime'])
planet_events.head()

,Year,Month,Date,Day,Time,Event,NaNdos,Datetime
0,2020,Jan,1,We,NaN,Venus: 34.6° E,NaN,2020-01-01
1,2020,Jan,4,Sa,08:38,Quadrantid Shower: ZHR = 120,NaN,2020-01-04
2,2020,Jan,5,Su,09:59,Perihelion: 0.9832 AU,NaN,2020-01-05
3,2020,Jan,10,Fr,15:01,Mercury Superior Conj.,NaN,2020-01-10
4,2020,Jan,10,Fr,19:10,Pen. Lunar Eclipse,NaN,2020-01-10


In [19]:
dummies = pd.get_dummies(planet_events['Event'])
dummy_cols = ['Mercury Superior Conj.', 'Mercury Inferior Conj.', 'Venus Inferior Conj.', 'Venus Superior Conj.','Uranus Conjunction', 'Neptune Conjunction', 'Saturn Conjunction', 'Jupiter Conjunction', 'Mars Conjunction', 'Neptune Opposition', 'Saturn Opposition', 'Jupiter Opposition', 'Uranus Opposition', 'Mars Opposition']
dummies = dummies[dummy_cols]
df['Datetime'] = pd.to_datetime(df['time'])
planet_events = pd.concat([planet_events, dummies], axis=1)
everything = pd.merge(df, planet_events, on='Datetime', how='left')
everything = everything.fillna(0)
everything

,time,ra,dec,Planet,CenterName,CenterLon,CenterLat,az,alt,dist (center to planet AU),...,Uranus Conjunction,Neptune Conjunction,Saturn Conjunction,Jupiter Conjunction,Mars Conjunction,Neptune Opposition,Saturn Opposition,Jupiter Opposition,Uranus Opposition,Mars Opposition
0,2000-Jan-01 00:00,18 04 55.50,-24 22 51.5,Mercury,Crowborough,0.15420000,51.0518512,17.74343784755384 deg2,-62.460788027094935 deg2,1.4130914988961565 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2000-Jan-02 00:00,18 11 45.95,-24 27 21.3,Mercury,Crowborough,0.15420000,51.0518512,16.374140650186238 deg2,-62.66790959130999 deg2,1.4177089536661651 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000-Jan-03 00:00,18 18 38.19,-24 30 32.3,Mercury,Crowborough,0.15420000,51.0518512,14.965991636726711 deg2,-62.84443967309148 deg2,1.421780515121529 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000-Jan-03 00:00,18 18 38.19,-24 30 32.3,Mercury,Crowborough,0.15420000,51.0518512,14.965991636726711 deg2,-62.84443967309148 deg2,1.421780515121529 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000-Jan-01 00:00,20 21 41.83,-19 12 59.3,Neptune,Crowborough,0.15420000,51.0518512,318.56401634757674 deg2,-52.43516868446793 deg2,31.021028239772164 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2000-Jan-02 00:00,20 21 50.64,-19 12 30.5,Neptune,Crowborough,0.15420000,51.0518512,319.8992216569468 deg2,-52.81749185318781 deg2,31.027733371983413 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2000-Jan-03 00:00,20 21 59.50,-19 12 01.6,Neptune,Crowborough,0.15420000,51.0518512,321.25414539949503 deg2,-53.18894639299184 deg2,31.034166715540692 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2000-Jan-03 00:00,20 21 59.50,-19 12 01.6,Neptune,Crowborough,0.15420000,51.0518512,321.25414539949503 deg2,-53.18894639299184 deg2,31.034166715540692 AU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
